In [21]:
%pip install pandas numpy matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json

Note: you may need to restart the kernel to use updated packages.


In [22]:
# DATASET_URL = "https://raw.githubusercontent.com/eric-minassian/rotten-tomatoes-dataset/main/movie_dataset.csv"
# df = pd.read_csv(DATASET_URL)
df = pd.read_csv("dataset.csv")

# Name the first column id
df.columns.values[0] = "id"

df.head()

,id,title,description,audience_score_average_rating,audience_score_liked_count,audience_score_not_liked_count,audience_score_review_count,audience_score_score,audience_score_sentiment,critic_score_average_rating,...,distributor,production_company,genre,sound_mix,rating,original_language,release_date_theater,release_date_streaming,runtime,box_office
0,rangreza,Rangreza,"Conflict arises when, Reshmi, a young woman wh...",3.9,4.0,1.0,2.0,80.0,POSITIVE,NaN,...,NaN,NaN,"['Romance', 'Musical', 'Drama']",NaN,NaN,Urdu,NaN,"Aug 1, 2018",2h 25m,NaN
1,yowamushi_pedal_the_movie,Yowamushi Pedal: The Movie,Sakamichi and the other Sohoku team members ra...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"['My Theater D.D., Inc.']","['Comedy', 'Adventure', 'Anime']",NaN,NaN,Japanese,NaN,"Oct 21, 2019",1h 30m,NaN
2,something_wicked_2014,Something Wicked,"When a young couple make their wedding plans, ...",2.2,11.0,52.0,14.0,17.0,NEGATIVE,NaN,...,NaN,NaN,"['Action', 'Mystery & Thriller']",NaN,R (Language|Violence|Sexuality),English,NaN,"Aug 29, 2016",1h 35m,NaN
3,conejo_en_la_luna,Rabbit on the Moon,A conspirator in an assassination implicates a...,3.3,32.0,25.0,20.0,56.0,NEGATIVE,NaN,...,NaN,['Instituto Mexicano de Cinematografía'],"['Crime', 'Drama', 'Mystery & Thriller']",['Dolby Digital'],NaN,English,NaN,NaN,1h 53m,NaN
4,1037599-voyager,Voyager,"Having recently survived a plane crash, Americ...",3.7,78.0,29.0,19.0,73.0,POSITIVE,5.8,...,['Castle Hill Productions'],"['Bioskop Film', 'Action Films']",['Drama'],['Surround'],PG-13,English,"Jan 31, 1992, Wide","Mar 12, 2020",1h 57m,NaN


In [23]:
# Summary of the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36072 entries, 0 to 36071
Data columns (total 28 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              36072 non-null  object 
 1   title                           36072 non-null  object 
 2   description                     33017 non-null  object 
 3   audience_score_average_rating   18939 non-null  float64
 4   audience_score_liked_count      18939 non-null  float64
 5   audience_score_not_liked_count  18939 non-null  float64
 6   audience_score_review_count     18939 non-null  float64
 7   audience_score_score            18939 non-null  float64
 8   audience_score_sentiment        18939 non-null  object 
 9   critic_score_average_rating     8600 non-null   float64
 10  critic_score_liked_count        8679 non-null   float64
 11  critic_score_not_liked_count    8679 non-null   float64
 12  critic_score_review_count       

In [24]:
# Percentage of missing values in each column
missing_values = df.isnull().mean() * 100

missing_values

id                                 0.000000
title                              0.000000
description                        8.469173
audience_score_average_rating     47.496673
audience_score_liked_count        47.496673
audience_score_not_liked_count    47.496673
audience_score_review_count       47.496673
audience_score_score              47.496673
audience_score_sentiment          47.496673
critic_score_average_rating       76.158794
critic_score_liked_count          75.939787
critic_score_not_liked_count      75.939787
critic_score_review_count         75.939787
critic_score_score                75.939787
critic_score_sentiment            75.939787
director                           2.425704
producer                          38.434243
screenwriter                      34.974495
distributor                       83.599468
production_company                50.277223
genre                              7.548791
sound_mix                         88.733644
rating                          